### Neural language models or how to write scientific papers

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import pandas as pd
from collections import Counter

In [ ]:
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz

--2021-03-26 02:39:51--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6030:18::a27d:5012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz [following]
--2021-03-26 02:39:52--  https://www.dropbox.com/s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc425e233d1959f12f17265d34e3.dl.dropboxusercontent.com/cd/0/get/BLZ9kVAug1Rpo9QmbzqyLKfdll2jr_wa16ea8GonWnj0--oAycd9pbFgUb7Xz27AeZFS-Ej1RgZsZo6ZyQqBTNhl_9Q-E08DoV3wrCupgeQ93oDWeSaHhqYFHNVUVCTzDVFgPO4U1ZU0fLqmrUE1pDIn/file?dl=1# [following]
--2021-03-26 02:39:52--  https://uc425e233d1959f12f17265d34e3.dl.dropboxusercontent.com/cd/0/get/BLZ9kVAug1Rpo9QmbzqyLKfdll2jr_wa16ea8GonWnj0--oAycd9pbFgUb7X

In [ ]:
data = pd.read_json("./arxivData.json")
data.sample(n=5)

,author,day,id,link,month,summary,tag,title,year
24486,"[{'name': 'Vladimir Kolmogorov'}, {'name': 'Ch...",14,1404.3538v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",4,The 38th Annual Workshop of the Austrian Assoc...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Proceedings of The 38th Annual Workshop of the...,2014
60,"[{'name': 'Baolin Peng'}, {'name': 'Kaisheng Y...",31,1506.00195v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,Recurrent Neural Networks (RNNs) have become i...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Recurrent Neural Networks with External Memory...,2015
38389,"[{'name': 'Shuhan Yuan'}, {'name': 'Panpan Zhe...",3,1706.00887v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",6,Wikipedia is the largest online encyclopedia t...,"[{'term': 'cs.CR', 'scheme': 'http://arxiv.org...",Wikipedia Vandal Early Detection: from User Be...,2017
1165,"[{'name': 'Tejas D. Kulkarni'}, {'name': 'Will...",11,1503.03167v4,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,This paper presents the Deep Convolution Inver...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Deep Convolutional Inverse Graphics Network,2015
3348,"[{'name': 'Tae-Hyun Oh'}, {'name': 'Yasuyuki M...",7,1512.02188v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",12,Commonly used in computer vision and other app...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Pseudo-Bayesian Robust PCA: Algorithms and Ana...,2015


In [ ]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'], axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

In [ ]:
SEQ_LEN = 4
BATCH_SIZE = 128
EPOCH = 2

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(
        self,
        lines,
        seq_length
    ):
    pass
    # your code goes here
    # do you need to change somehow initialization of tokens for LSTM?

    def load_words(self):
        train_df = pd.DataFrame(self.lines, columns=['text'])
        text = train_df['text'].str.cat(sep=' ')
        return text.split(' ')

    def get_uniq_words(self):
        word_counts = Counter(self.words)
        return sorted(word_counts, key=word_counts.get, reverse=True)

    def __len__(self):
        return len(self.words_indexes) - self.seq_length

    def __getitem__(self, index):
        return (
            torch.tensor(self.words_indexes[index:index+self.seq_length]),
            torch.tensor(self.words_indexes[index+1:index+self.seq_length+1]),
        )

In [ ]:
import torch
from torch import nn

class LSTMModel(nn.Module):
    def __init__(self, dataset):
        pass
        # your code goes here

In [ ]:
dataset = Dataset(lines, SEQ_LEN)

In [ ]:
model = Model(dataset)
model = model.to(device)

In [ ]:
import argparse
import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader

def train(dataset, model):
    model.train()

    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, patience=10, verbose=True)

    for epoch in range(EPOCH):
        state_h, state_c = model.init_state(4)

        state_h = state_h.to(device)
        state_c = state_c.to(device)
        batch_loss = []
        for batch, (x, y) in enumerate(dataloader):
            optimizer.zero_grad()

            x = x.to(device)
            y = y.to(device)

            y_pred, (state_h, state_c) = model(x, (state_h, state_c))
            loss = criterion(y_pred.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss.backward()
            optimizer.step()
            batch_loss.append(loss.item())
            if batch % 100 == 0:
                avg_loss = np.mean(batch_loss)
                batch_loss = []
                scheduler.step(avg_loss)
                print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item(), 'average loss:': avg_loss })

In [ ]:
def predict(dataset, model, text, next_words=100):
    model.eval()

    words = text.split(' ')
    state_h, state_c = model.init_state(len(words))

    state_h = state_h.to(device)
    state_c = state_c.to(device)

    with torch.no_grad():
        for i in range(0, next_words):
            x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]])
            x = x.to(device)
            y_pred, (state_h, state_c) = model(x, (state_h, state_c))

            last_word_logits = y_pred[0][-1]
            p = torch.nn.functional.softmax(last_word_logits, dim=0).cpu().detach().numpy()
            word_index = np.random.choice(len(last_word_logits), p=p)
            words.append(dataset.index_to_word[word_index])

    return ' '.join(words)

In [ ]:
# train(dataset, model)

In [1]:
# print(predict(dataset, model, text='AI'))